# get reviews

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#cd /content/drive/MyDrive/Project/letterboxd

/content/drive/MyDrive/Project/letterboxd


In [1]:
import requests, time, json, re
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
# Disable chardet debug messages
import logging
logging.getLogger().setLevel(logging.WARNING)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from metadata import *

import multiprocessing
cpu_count = multiprocessing.cpu_count()

import os
os.getcwd()

'/home/jupyter/movie-analysis'

In [2]:
!pip install pandarallel -qq

In [3]:
from pandarallel import pandarallel
print(cpu_count)
pandarallel.initialize(nb_workers=cpu_count-1, progress_bar=True, use_memory_fs=False)

96
INFO: Pandarallel will run on 95 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## check num of reviews

In [ ]:
def get_all(row):
    film = row.film
    return movieReview(film).get_num_reviews()

### KR

In [ ]:
kr=pd.read_pickle('data/kr_meta.pkl')

In [ ]:
%%time
kr['num_reviews'] = kr.parallel_apply(get_all, axis=1)

Sleep...
Sleep...
CPU times: user 13.4 s, sys: 3.68 s, total: 17.1 s
Wall time: 3min 41s


In [ ]:
kr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11651 entries, 0 to 11650
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         11651 non-null  object 
 1   rate         1536 non-null   float64
 2   meta         11651 non-null  object 
 3   num_reviews  11651 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 364.2+ KB


In [ ]:
kr.to_pickle('data/kr_meta.pkl')

### CN

In [ ]:
cn=pd.read_pickle('data/cn_meta.pkl')

In [ ]:
%%time
cn['num_reviews'] = cn.parallel_apply(get_all, axis=1)

CPU times: user 31.9 s, sys: 8.57 s, total: 40.4 s
Wall time: 8min 56s


In [ ]:
cn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28304 entries, 0 to 28303
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         28304 non-null  object 
 1   rate         1691 non-null   float64
 2   meta         28304 non-null  object 
 3   num_reviews  28304 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 884.6+ KB


In [ ]:
cn.to_pickle('data/cn_meta.pkl')

### JP

In [ ]:
jp=pd.read_pickle('data/jp_meta.pkl')

In [ ]:
%%time
jp['num_reviews'] = jp.parallel_apply(get_all, axis=1)

Sleep...
Sleep...
CPU times: user 39.8 s, sys: 11.1 s, total: 50.8 s
Wall time: 10min 19s


In [ ]:
jp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34689 entries, 0 to 34688
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         34689 non-null  object 
 1   rate         4005 non-null   float64
 2   meta         34689 non-null  object 
 3   num_reviews  34689 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB


In [ ]:
jp.to_pickle('data/jp_meta.pkl')

## get reviews' links for non-zero films

### KR

In [4]:
kr=pd.read_pickle('data/kr_meta.pkl')
kr1 = kr[kr.num_reviews >= 5].reset_index()
kr1.shape, kr1.num_reviews.max()

((3104, 5), 233868)

In [ ]:
get_all_reviews

In [5]:
%%time
kr1['reviews'] = kr1.parallel_apply(get_all_reviews, axis=1)

sleep...
sleep...
sleep...
sleep...
CPU times: user 4.2 s, sys: 2.33 s, total: 6.53 s
Wall time: 8min 33s


In [6]:
kr1['len'] = [len(r) for r in kr1['reviews']]
kr1[(kr1.len != kr1.num_reviews) & (kr1.len != 3072)]

,index,film,rate,meta,num_reviews,reviews,len
13,197,/film/i-am-a-truck/,NaN,"{'title': 'I Am a Truck', 'year': 1953.0, 'dir...",8,"[{'rating': '6', 'link': '/drlemon/film/i-am-a...",9
82,585,/film/the-flower-girl/,3.37,"{'title': 'The Flower Girl', 'year': 1972.0, '...",89,"[{'rating': '8', 'link': '/supremelemon/film/t...",90
85,633,/film/returned-a-single-legged-man/,NaN,"{'title': 'Returned A Single-Legged Man', 'yea...",11,"[{'rating': '4', 'link': '/dragon_needles/film...",10
217,1662,/film/the-moon-is-the-suns-dream/,2.51,"{'title': 'The Moon Is... the Sun's Dream', 'y...",522,"[{'rating': '4', 'link': '/annanomaly/film/the...",526
264,1984,/film/trio-1997/,2.77,"{'title': 'Trio', 'year': 1997.0, 'director': ...",327,"[{'rating': '4', 'link': '/annanomaly/film/tri...",331
...,...,...,...,...,...,...,...
3090,11200,/film/black-knight-2023/,3.39,"{'title': 'Black Knight', 'year': 2023.0, 'dir...",376,"[{'rating': '5', 'link': '/annodomini/film/bla...",377
3094,11212,/film/the-roundup-no-way-out/,3.29,"{'title': 'The Roundup: No Way Out', 'year': 2...",278,[{'link': '/anuragkashyap/film/the-roundup-no-...,280
3100,11220,/film/bitch-x-rich/,NaN,"{'title': 'BITCH X RICH', 'year': 2023.0, 'dir...",25,"[{'link': '/yeritao/film/bitch-x-rich/', 'date...",38
3102,11228,/film/the-childe/,3.30,"{'title': 'The Childe', 'year': 2023.0, 'direc...",227,"[{'rating': '8', 'link': '/djaycoholyc/film/th...",236


In [9]:
kr1 = kr1.drop(['index','len'],axis=1)
kr1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3104 entries, 0 to 3103
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         3104 non-null   object 
 1   rate         1534 non-null   float64
 2   meta         3104 non-null   object 
 3   num_reviews  3104 non-null   int64  
 4   reviews      3104 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 121.4+ KB


In [10]:
kr1.to_pickle('data/kr_reviews.pkl')

### CN

In [11]:
cn=pd.read_pickle('data/cn_meta.pkl')
cn1 = cn[cn.num_reviews >= 5].reset_index()
cn1.shape

(5751, 5)

In [12]:
%%time
cn1['reviews'] = cn1.parallel_apply(get_all_reviews, axis=1)

sleep...
sleep...
sleep...
sleep...
sleep...
CPU times: user 6.24 s, sys: 3.73 s, total: 9.97 s
Wall time: 8min 8s


In [16]:
cn1['len'] = [len(r) for r in cn1['reviews']]
cn1[(cn1.len != cn1.num_reviews) & (cn1.len != 3072)].shape

(237, 7)

In [21]:
cn1[cn1.len == 3072].num_reviews.min()

3081

In [22]:
cn1 = cn1.drop(['index','len'],axis=1)
cn1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5751 entries, 0 to 5750
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         5751 non-null   object 
 1   rate         1690 non-null   float64
 2   meta         5751 non-null   object 
 3   num_reviews  5751 non-null   int64  
 4   reviews      5751 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 224.8+ KB


In [23]:
cn1.to_pickle('data/cn_reviews.pkl')

### JP

In [24]:
jp=pd.read_pickle('data/jp_meta.pkl')
jp1 = jp[jp.num_reviews >= 5].reset_index()
jp1.shape

(10179, 5)

In [25]:
%%time
jp1['reviews'] = jp1.parallel_apply(get_all_reviews, axis=1)

sleep...
sleep...
sleep...
sleep...
sleep...
sleep...
CPU times: user 8.84 s, sys: 6.8 s, total: 15.6 s
Wall time: 21min 56s


In [27]:
jp1['len'] = [len(r) for r in jp1['reviews']]
jp1[(jp1.len != jp1.num_reviews) & (jp1.len != 3072)]

,index,film,rate,meta,num_reviews,reviews,len
3,8,/film/katsudo-shashin/,3.07,"{'title': 'Katsudō Shashin', 'year': 1907.0, '...",343,"[{'rating': '10', 'link': '/dumbsville/film/ka...",344
5,34,/film/the-dull-sword/,3.10,"{'title': 'The Dull Sword', 'year': 1917.0, 'd...",115,"[{'link': '/mpj/film/the-dull-sword/', 'date':...",117
22,134,/film/a-page-of-madness/,3.82,"{'title': 'A Page of Madness', 'year': 1926.0,...",2920,"[{'rating': '8', 'link': '/kurstboy/film/a-pag...",2923
87,368,/film/i-was-born-but/,3.97,"{'title': 'I Was Born, But...', 'year': 1932.0...",1544,"[{'rating': '8', 'link': '/oneaprilday/film/i-...",1545
132,489,/film/the-actress-and-the-poet/,NaN,"{'title': 'The Actress and the Poet', 'year': ...",36,[{'link': '/fuchsiadyke/film/the-actress-and-t...,37
...,...,...,...,...,...,...,...
10162,34035,/film/old-narcissus-2023/,NaN,"{'title': 'Old Narcissus', 'year': 2023.0, 'di...",9,"[{'rating': '5', 'link': '/billhsu/film/old-na...",10
10163,34040,/film/monster-2023/,3.97,"{'title': 'Monster', 'year': 2023.0, 'director...",745,"[{'rating': '7', 'link': '/davidehrlich/film/m...",757
10165,34045,/film/geats-extra-kamen-rider-punkjack/,NaN,"{'title': 'Geats Extra: Kamen Rider PunkJack',...",16,[{'link': '/kreideprinz/film/geats-extra-kamen...,17
10177,34088,/film/black-clover-sword-of-the-wizard-king/,3.51,{'title': 'Black Clover: Sword of the Wizard K...,1403,"[{'rating': '8', 'link': '/emanuele2002/film/b...",1416


In [28]:
jp1[jp1.len == 3072].num_reviews.min()

3088

In [29]:
jp1 = jp1.drop(['index','len'],axis=1)
jp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10179 entries, 0 to 10178
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         10179 non-null  object 
 1   rate         4003 non-null   float64
 2   meta         10179 non-null  object 
 3   num_reviews  10179 non-null  int64  
 4   reviews      10179 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 397.7+ KB


In [30]:
jp1.to_pickle('data/jp_reviews.pkl')

In [34]:
from google.cloud import storage
import pickle
storage_client = storage.Client()

bucket = storage_client.bucket('movie-analysis')
blob = bucket.blob('kr_reviews.pkl')

# write
with blob.open(mode='wb') as f:
    pickle.dump(kr1, f)

# read
# with blob.open(mode='rb') as f:
#     d_reloaded = pickle.load(f)


# get review contents